In [0]:
%python
import requests
from bs4 import BeautifulSoup
import pandas as pd
from psycopg2.extensions import AsIs
from psycopg2 import extras
from psycopg2 import sql
from sqlalchemy import create_engine
from datetime import datetime

In [0]:
%python
def get_companies_listed_on_wiener_borse():
    """
     reads all companies with isin number that can be traded at the vienna stock exchange
     paramters: None
     return: dataFrame with companies
    """
    url_companies = "https://www.wienerborse.at/listing/aktien/unternehmensliste/"
    response = requests.get(url_companies)
    soup = BeautifulSoup(response.text, 'html.parser')
    tags = soup.find_all("tr")
    number_companies = int(tags[-1].attrs["data-key"])
    stock_companies = pd.DataFrame({"isin" : [], "name" : [], "country" : []})
    for l in range(number_companies+1):
        result = soup.findAll("tr", {"data-key" : l})
        isin = result[0].contents[0].text
        name = result[0].contents[1].text
        country = result[0].contents[2].text
        stock_companies = stock_companies.append({"isin" : isin, "name" : name, "country" : country}, ignore_index=True)
    return stock_companies

In [0]:
%python
def remove_punctuation(text):
    """
    removes punctuation from a string and makes all lower case
    paramters: string
    return: same sting but all smaller case, no punctuation and no redundant whitespaces 
    """
    text = re.sub(r'_',' ',text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'[\n]',' ',text)
    return text.lower().lstrip()

In [0]:
%python
def read_keywords_wordCount():
    """
    reads and provides a list of keywords that should be displayed (e.g., in a word count)
    paramters: None
    return: list of keywords
    """
    path = "/dbfs/FileStore/shared_uploads/ds21m031@codingmohgmail.onmicrosoft.com/ds_keywords.txt"
    with open(path, 'r') as f:
        lines = f.read()
    return re.findall(r'[a-z]+',lines)

In [0]:
%python
def do_wordCount(x, keywords):
    """
    performs word count on an rdd 
    paramters: an df with multiple strings, set of relevant keywords
    return: pandas Dataframe in which each word is associated with the number it was encountered
    """
    return     (x.rdd
                .flatMap(lambda x: x[0].split())
                .filter(lambda x: x != "")
                .map(lambda x: (x,1))
                .reduceByKey(lambda a,b : a+b) 
                .filter(lambda x: x[0] in keywords)
                .toDF()
                .toPandas()
                .rename(columns={'_1':'word', '_2' : 'count'})
                )

In [0]:
%python
def transform_data_lm(data):
    """ Transforms pyspark dataframe for use in LM"""
#    return data.rdd.map(lambda r: [Vectors.dense(r[:-2]), r[-1] ]).toDF(['features','label'])
    return data.rdd.map(lambda r: [Vectors.dense(r[0]), r[1] ]).toDF(['features','label'])

In [0]:
%python
def calculate_LM_per_index(data):
    """ 
    Calculates a linear model for each index in data set
    Input: pyspark data frame - cloumns are indexes; last two columns are for the date in datetime and int format
    Output: Pandas DF with LM coefficients per index    
    """
    stock_data = pd.DataFrame({"isin" : [], "coefficient" : [], "intersept" : []})
    lr = LinearRegression(fitIntercept=True, maxIter = 100)

    for col in data.columns[:-2]:
        """For each index calculate linear model"""
    #    print(col)
        dataLM = transform_data_lm( data.select(col, "date_ts") )
        lrModel = lr.fit(dataLM)
        # Turn coordination system
        stock_data = stock_data.append({"isin" : col , "coefficient" : 1 / lrModel.coefficients.values[0], "intersept" : - lrModel.intercept / lrModel.coefficients.values[0]} , ignore_index=True)
    #    print("Coefficients: %s" % str(lrModel.coefficients))
    #    print("Intercept: %s" % str(lrModel.intercept))
    #    summary = lrModel.summary
    #    print("RMSE: %f" % summary.rootMeanSquaredError)
    #    print("r2: %f" % summary.r2)
    return stock_data

In [0]:
%python

def adjust_string_for_matching(word):
    """
    Takes a string as input and manupulates it for a more efficient matching of 
    company names
    """
    word = word.lower()
    word = re.sub(r'international','',word)
    word = re.sub(r'pharmaceuticals','',word)
    word = re.sub(r'pharma','',word)
    word = re.sub(r' ag$','',word)
    word = re.sub(r' se$','',word)
    word = re.sub(r'group','',word)
    word = re.sub(r'corporation','',word)
    word = re.sub(r'inc\.{1}','',word)
    return word
    
    

In [0]:
%python

def find_best_match(company, possibleMatches, cut_off_score): 
    """
    for a given company (string) this function provides the best match from the list possibleMatches
    input: company (string), possibleMatches list of strings 
    output: string from possibleMathes with the best similarity score
    """
 #   matches = [(i, fuzz.ratio(company.iloc[0].lower(), i.lower())) for i in possibleMatches ]
    matches2 = [(i, fuzz.ratio(adjust_string_for_matching(company.iloc[0]), adjust_string_for_matching(i))) for i in possibleMatches ]
    
 #   bestFit = max(matches,key= lambda x: x[1])
    bestFit2 = max(matches2,key= lambda x: x[1])
    if bestFit2[1] > cut_off_score:
  #      print("input {}, match_1 {}, score_1 {}, match_2 {}, score_2 {}".format(company.iloc[0], bestFit[0], bestFit[1], bestFit2[0], bestFit2[1]))
        print("input {}, match {}, score {}".format(company.iloc[0], bestFit2[0], bestFit2[1]))
        return bestFit2[0]
    else:
        return ""

In [0]:
%python
def geo_coder(address):
    """
    geo code job location
    input: address string
    output longitude and latitude
    !!max number of requests is limited
    """
    url = 'http://open.mapquestapi.com/geocoding/v1/address'
    params = {'key': ' 	XHd8D8B7BHYI9MPhOYmJ36YAlhNegD4J', 'location': address}
    r = requests.get(url, params=params)
    r2 = r.json()["results"][0]["locations"][0]['latLng']
    return [r2['lng'], r2['lat']]

In [0]:
%python

def read_stock_data(indexList, start, end, ss):
    """
    reads stock data from yfinance api
    input: list of relevant isin numbers, start date, end date, spark session
    output: pyspark dataframe in the provided spark session
    """
    df = yf.download(indexList, start, end)["Close"] 
    df["date"] = df.index
    return ss.createDataFrame(df)

In [0]:
%python
def read_list_aktienfinder():
    """
     reads csv from aktienfinder
     paramters: None
     return: dataFrame with companies
    """
    path = "/dbfs/FileStore/shared_uploads/ds21m031@codingmohgmail.onmicrosoft.com/companies_aktienfinder.csv"
    return pd.read_csv(path, names = ["isin", "name"])

In [0]:
%python
def write_to_postgres(df):
    """
    no jdbc driver available on databricks
    """
    (df.write.format("jdbc")
     .option("url", "jdbc:postgresql://dsc-inf.postgres.database.azure.com:5432/nyt_import")
     .option("driver", "org.postgresql.Driver")
     .option("dbtable", "ds21_b1_jobs_result")
     .option("user", "ds21m031")
     .option("password", "surf1234")
     .option("mode", "overwrite")
     .save()
    )

In [0]:
%python
def insert_job_result_to_postgres(df, params, tableName):
    """ 
    insert result of job analysis in to postgres db 
    input pyspark dataframe
    """
    
    print(tableName)
    sql_insert = '''INSERT INTO %s 
            (company_name_ad ,
            company_name_stock ,
            isin ,
            description ,
            description_clean ,
            title ,
            salary ,
            schedule ,
            date_loaded ,
            lm_coefficint ,
            lm_intersept ,
            city ,
            country ,
            longitude ,
            latitude ,
            platform)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
    sql_drop =   "DROP TABLE IF EXISTS %s;"
    sql_create = ''' CREATE TABLE IF NOT EXISTS %s   
            (id serial UNIQUE, 
            company_name_ad text,
            company_name_stock text,
            isin text,
            description text,
            description_clean text,
            title text,
            salary text,
            schedule text,
            date_loaded date,
            lm_coefficint real,
            lm_intersept real,
            city text,
            country text,
            longitude real,
            latitude real,
            platform text, 
            created_at TIMESTAMPTZ NOT NULL DEFAULT NOW());
        '''
    
    conn = None
    try:
        conn = psycopg2.connect(host=params['host'],
                        port=params['port'],
                        database=params['database'],
                        user=params["user"],
                        password=params["pass"]) # connect to the PostgreSQL database
        cur = conn.cursor()               # create a new cursor 
        cur.execute(sql_drop, (AsIs(tableName),))
        cur.execute(sql_create, (AsIs(tableName),))

        
        for row in df.collect(): #!!!!!!!!!!!! TEMPORARY SOLUTION AS DRIVER NOR AVAILABLE
            tup = [i for i in row]
            tup.insert(0, AsIs(tableName))
            cur.execute(sql_insert, tup)

        conn.commit()                     # commit the changes to the database
        cur.close()                       # close communication with the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
            
           

In [0]:
%python
def insert_word_count_to_postgres(df_list, params, tableName):
    """ 
    insert result of word count into postgres db 
    input list of pandas dataframes, paramters for connecting to db, tablename 
    """
    url = f"postgresql://{params['user']}:{params['pass']}@{params['host']}:{params['port']}/{params['database']}"
    engine = create_engine(url)
    
    sql_drop =   "DROP TABLE IF EXISTS %s;"
    engine.execute(sql_drop, (AsIs(tableName), ))
    
    
    for table in df_list:
        table['Timestamp'] = datetime.now()
        table.to_sql(tableName, engine, if_exists = "append", index = False)
    